In [2]:
import ollama
import psycopg2


In [3]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from sentence_transformers import SentenceTransformer


conn = psycopg2.connect(
    dbname="mydb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()


embedder = SentenceTransformer("BAAI/bge-m3")


def add_document(text):
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()


base_url = "https://www.agnoshealth.com/forums/search?page="
page = 1

while True:
    url = base_url + str(page)
    web_data = requests.get(url)
    soup = BeautifulSoup(web_data.text, 'html.parser')

    
    find_word = soup.find_all("div", class_="flex flex-col")

  
    if not find_word:
        print("No data:", page)
        break

    for i in find_word:
        text = i.get_text(strip=True)
        print(text, "\n")
        add_document(text)

    page += 1
   


cur.close()
conn.close()

def generate_response(query_text):
    retrieved_docs = query_postgresql(query_text)
    context = "\n".join([doc[0] for doc in retrieved_docs])
    prompt = f"Answer the question:\n{context}\n\nQuestion:{query_text}"
    response = ollama.chat(model="llama3.2",messages=[
        {"role":"system","content":"You are a kind, informative AI medical assistant"},
        {"role":"user","content":prompt}
    ])
    return response["message"]["content"]

C:\Users\PC-Ton\AgnosTask1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


หญิง|อายุ23ปีเสี่ยงต่อโรคซึมเศร้า (Major depressive disorder)2/18/2024เสี่ยงต่อโรคซึมเศร้าอ่อนเพลียไม่มีแรง+3สวัสดีค่ะ  อยากจะปรึกษาค่ะในช่วง2-3วันที่ผ่านมามีความรู้สึกดาวน์มากๆ รู้สึกเบื่อ ไม่อยากทำอะไร ไม่อยากเจอใคร ใครพูดอะไรก็หงุดหงิดไปหมด บางครั้งอยู่ดีๆก็รู้สึกแย่จนร้องไห้ก็มี นอนไม่หลับด้วยค่ะ  กว่าจะหลับใช้เวลานานมาก  มาวันนี้ไม่รู้สึกดาวน์แล้วค่ะ  แต่มันเหนื่อย เพลีย  รู้สึกอยากนอนตลอดเวลา  นี่ไม่ใช่ครั้งแรกที่เป็นค่ะ  เป็นทุกครั้งที่เครียดมากๆ หรือมีความรู้สึกแย่ๆเข้ามา  กว่าจะหายต้องใช้เวลาเป็นอาทิตย์เลยค่ะ มันเป็นเพระเครียดมากจนเกินไปหรือเพราะอะไรคะ  แล้วควรไปพบแพทย์เพื่อรับการรักษาดีไหมคะ 

ชาย|อายุ21ปีกระเพาะปัสสาวะอักเสบ (Cystitis)2/16/2024กระเพาะปัสสาวะอักเสบปัสสาวะกะปริบกะปรอยปัสสาวะบ่อย+1สวัสดีครับคือว่าผมมีอาการนี้มาได้ปีสองปีแล้วครับ เคยรับการรักษาแล้วเหมือนได้ทานยาระงับการบีบตัวกระเพาะปัสสาวะอยู่ช่วงนึง แต่ไม่รู้สึกว่าดีขึ้นแล้วก็ไม่ได้ไปรับการรักษาอีกเลยจนมันชินไปเอง ปวดปัสสาวะบ่อยๆ แต่จำนวนไม่เยอะ แต่เหมือนตอนนี้เข้าห้องน้ำบ่อยๆ จะเป็นปัญหามากขึ้นแล้วกลัวว่าจะเป็

In [6]:
conn = psycopg2.connect(
    dbname="mydb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT,
        embedding vector(1024)
    );
""")
conn.commit()
cur.close()
conn.close()


In [ ]:
conn = psycopg2.connect(
    dbname = "mydb",
    user = "admin",
    password = "1234",
    host = "localhost",
    port = "5432"
    
)
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    content TEXT,
    embedding vector(1024
""")
conn.commit()
cur.close( )
conn.close()


In [7]:
def query_postgresql(query_text, k=5):
    query_embedding = embedder.encode(query_text).tolist()
    conn = psycopg2.connect(
        dbname="mydb",
        user="admin",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    query_embedding_str = "[" + ",".join(map(str, query_embedding)) + "]"
    sql_query = """
        SELECT content,embedding <=> %s::vector AS similarity_score
        FROM documents
        ORDER BY similarity_score ASC
        LIMIT %s;
    """
    cur.execute(sql_query, (query_embedding_str, k))
    results = cur.fetchall()
    conn.close()
    return results


query_postgresql("ขอดูคนไข้อายุ 19")


[('หญิง|อายุ19ปีผื่นตามร่างกาย (Dermatitis unspecified)9/4/2024ผื่นตามร่างกายอยากรู้ว่าเป็นโรคอะไรและวิธีการรักษาค่ะ',
  0.3975304602104689),
 ('หญิง|อายุ19ปีเสี่ยงต่อโรคซึมเศร้า (Major depressive disorder)1/15/2023รู้สึกไร้ค่าอยากตายทำร้ายตัวเอง+4รู้สึกเหนื่อยท้อแท้ไม่อยากใช้ชีวิตนอนไม่หลับมีเรื่องครอบครัวกระทบจิตใจตั้งแต่อายุ15จนตอนนี้อายุ19แล้วค่ะควรไปพบแพทย์มั้ยคะแล้วหนูต้องเริ่มยังไงบ้าง',
  0.40765925691480875),
 ('หญิง|อายุ19ปีเสี่ยงต่อโรคอารมณ์สองขั้ว (Bipolar disorder)10/20/2022เสี่ยงต่อโรคอารมณ์สองขั้วเศร้าท้อแท้+7ควรเริ่มรักษายังไงค่ะ',
  0.4206203723693338),
 ('ชาย|อายุ19ปีเสี่ยงต่อโรคซึมเศร้า (Major depressive disorder)8/29/2023ง่วงตลอดเวลาหลับมากกว่าปกติรู้สึกไร้ค่า+6ตัองทำยังไงต่อไป',
  0.42226442231323746),
 ('ชาย|อายุ19ปีปวดศีรษะไมเกรน (Migraine)10/8/2023ปวดศีรษะไมเกรนปวดหัวผมมีอาการแสบร้อนบนศรีษะครับเกิดจากอะไรครับ',
  0.42236962451753657)]

In [10]:
def generate_response(query_text):
    retrieved_docs = query_postgresql(query_text)
    context = "\n".join([doc[0] for doc in retrieved_docs])
    prompt = f"Answer the question:\n{context}\n\nQuestion:{query_text}"
    response = ollama.chat(model="llama3.2",messages=[
        {"role":"system","content":"You are a professional and ethical medical doctor who provides accurate, safe, and easy-to-understand health advice to patients. When answering questions, you should use the patient’s medical records."},
        {"role":"user","content":prompt}
    ])
    return response["message"]["content"]

generate_response("ปวดศีรษะไมเกรน")


'ขออภัยสำหรับคำถามที่ไม่ชัดเจน คุณต้องการทราบวิธีการรักษาหรือการบำบัดสำหรับปวดศีรษะไมเกรนหรือไม่? หากคุณเป็นคนป่วยกับปวดศีรษะไมเกรนแล้วสามารถให้คำแนะนำและข้อควรระวังเพิ่มเติมได้ดังนี้\n\n1.  **การรักษาด้วยยา**: ยาไมเกรนมักจะถูกใช้เป็นยารักษาโรค แต่การใช้ยาห้ามต้องมีการควบคุมจากแพทย์ เพื่อหลีกเลี่ยงผลข้างเคียงและปัญหาทางการแพทย์\n2.  **การเปลี่ยนแปลงนิสัย**: การปรับเปลี่ยนนิสัย เช่น การหลีกเลี่ยงสาเหตุการเกิดไมเกรน เช่น เสียงดัง, สีสว่าง, อุณหภูมิ, หรือการกินอาหารที่มีส่วนผสมของแอนตี้เส้นใย\n3.  **การออกกำลังกาย**: การออกกำลังกายเป็นวิธีที่ดีในการลดความเครียดและปรับปรุงสุขภาพโดยรวม\n4.  **การนอนหลับให้เพียงพอ**: การนอนหลับให้เพียงพอสามารถช่วยให้คุณรู้สึกสดชื่นและมีสุขภาพดี\n5.  **การเปลี่ยนแปลงอาหาร**: การกินอาหารที่มีคุณภาพและสมดุล สามารถช่วยลดความเครียดและปรับปรุงสุขภาพโดยรวม\n\nหากคุณเป็นคนป่วยกับปวดศีรษะไมเกรน ควรปรึกษาแพทย์หรือผู้เชี่ยวชาญเพื่อให้ได้รับการรักษาที่เหมาะสมตามอาการของคุณ'